In [1]:
from __future__ import division

import pandas as pd
import numpy as np
import scipy as sp

import scipy.stats
import matplotlib.pyplot as plt
import toyplot as tp

### Множественная проверка гипотез

Задача ставится следующим образом: у нас есть набор наблюдений $1,...,n$, для каждого из которых мы сформулировали нулевую гипотезу. Далее, возможны 2 варианта:
- мы хотим одновременно проверить все гипотезы (Global hypothesis testing);
- мы хотим проверить каждую из $n$ гипотез в отдельности (Multiple testing).

### Одновременная проверка гипотез (Global hypothesis testing)

** Тест Фишера **

В основе лежит следующая идея: мы имеем на руках набор из $n$ p-values. Если сделать предположение о том, что они равномерно распределены на отрезке $[0, 1]$, то $-log(p_i) \sim Exp(1)$.

Тестовая статитистика имеет вид: 
$$ T = -2 \sum_{i=1}^{n} log \, p_{i}.$$

Распределение статистики - $\chi^2_{2n}$.

### Последовательная проверка гипотез (Multiple hypothesis testing)

Отклонение нулевой гипотезы - это хорошо, однако нам также хотелось бы знать ,какие именно нулевые гипотезы не прошли проверку. Тут и используется последовательная проверка гипотез.

** Метод Бонферрони **

Метод достаточно прост: мы рассчитываем p-value для каждой гипотезы в отдельности и отвергаем "глобальную" нулевую гипотезу только в том случае, если минимальное p-value меньше чем $\alpha/n.$

** Метод Бенжамина-Хокберга (Benjamin and Hochberg) **

Для понимания сути метода нам придется ввести такое понятие как FDP (false discovery proportion):
$$ FDP = \frac{V}{R},$$
где $V$ - число ложных отклонений, $R$ - общее число отклонений.
Метод позволяет контроллировать ожидание $FDP$, так чтобы оно было ниже некоторой априорной величины False Discovery Rate:
$$ \frac{E[V]}{max(R,1)} <q. $$

Процедуру можно описать следующим образом:
- отсортировать все p-values по возрастанию;
- найти  $p_i : p_i \leq q\frac{i}{n};$
- отклонить все гипотезы до $p_i.$

### Пример

В нашем наборе данных у нас есть только фамилия игрока. Чтобы исключить группировку для однофамильцев, мы в качестве уникального ключа будем считать пару "фамилия - имя команды".

In [2]:
df = pd.read_csv('free_throws.csv', names=["away", "home", "team", "player", "score"])
df["at_home"] = df["home"] == df["team"]
df.head()

,away,home,team,player,score,at_home
0,HOU,LAL,LAL,Bryant,0,True
1,HOU,LAL,LAL,Bryant,1,True
2,HOU,LAL,LAL,Fisher,1,True
3,HOU,LAL,LAL,Fisher,1,True
4,HOU,LAL,LAL,Bryant,1,True


Сгруппируем данные по факту игры "дома".

In [3]:
df.groupby("at_home").mean()

,score
at_home,
False,0.759678
True,0.761635


Пользуясь средствами pandas, можно легко получить статистику по каждому из игроков:

In [4]:
sdf = pd.pivot_table(df, index=["player", "team"], columns="at_home", values=["score"], 
               aggfunc=[len, sum], fill_value=0).reset_index()
sdf.columns = ['player', 'team', 'atm_away', 'atm_home', 'score_away', 'score_home']

sdf['atm_total'] = sdf['atm_away'] + sdf['atm_home']
sdf['score_total'] = sdf['score_away'] + sdf['score_home']

sdf.head(10)

,player,team,atm_away,atm_home,score_away,score_home,atm_total,score_total
0,A. Brown,CHA,0,2,0,2,2,2
1,A. Brown,MEM,16,6,7,3,22,10
2,A. Johnson,ATL,14,18,12,15,32,27
3,A. Johnson,TOR,63,57,44,38,120,82
4,Abdur-Rahim,SAC,4,0,4,0,4,4
5,Acker,DET,2,0,1,0,2,1
6,Acker,LAC,6,2,2,2,8,4
7,Adams,TOR,6,0,3,0,6,3
8,Adrien,GSW,12,7,7,4,19,11
9,Adrien,HOU,6,6,3,4,12,7


Теперь можем применить уже знакомый нам тест о значении доли для каждого из $n$ игроков - $p_i$.

Нулевая гипотеза примет вид: не важно, играет ли команда дома или гостях, ожидаемое число попаданий для каждого игрока постоянно. Вид нулевой гипотезы для каждого из игроков нам известен:

$$ H_{0,i}: p_{i,h} = p_{i,a}, $$
$$ H_{1,i}: p_{i,h} \neq p_{i,a}. $$

Тестовая статистика имеет вид:

$$Z = \frac{\hat{p}_h-\hat{p}_a}{\sqrt{\hat{p}(1-\hat{p})(\frac{1}{n_h}+\frac{1}{n_a})}}.$$

Далее, оставим игроков, для которых есть хотя бы 30 наблюдений:

In [5]:
data = sdf.query('atm_total > 30').copy()
len(data)

957

In [6]:
data['p_home'] = data['score_home'] / data['atm_home']
data['p_away'] = data['score_away'] / data['atm_away']
data['p_ovr'] = (data['score_total']) / (data['atm_total'])

# two-sided
data['zval'] = (data['p_home'] - data['p_away']) / np.sqrt(data['p_ovr'] * (1-data['p_ovr']) * (1/data['atm_away'] + 1/data['atm_home']))
data['pval'] = 2*(1-sp.stats.norm.cdf(np.abs(data['zval'])))

In [7]:
data.head(10)

,player,team,atm_away,atm_home,score_away,score_home,atm_total,score_total,p_home,p_away,p_ovr,zval,pval
2,A. Johnson,ATL,14,18,12,15,32,27,0.833333,0.857143,0.843750,-0.184017,0.854000
3,A. Johnson,TOR,63,57,44,38,120,82,0.666667,0.698413,0.683333,-0.373327,0.708905
10,Afflalo,DEN,227,274,182,218,501,400,0.795620,0.801762,0.798403,-0.170570,0.864562
11,Afflalo,DET,72,86,61,65,158,126,0.755814,0.847222,0.797468,-1.423866,0.154485
14,Ahearn,MIA,19,12,18,12,31,30,1.000000,0.947368,0.967742,0.807856,0.419173
21,Aldridge,POR,797,839,606,671,1636,1277,0.799762,0.760351,0.780562,1.925169,0.054208
23,Alexander,MIL,36,47,24,34,83,58,0.723404,0.666667,0.698795,0.558375,0.576588
24,Allen,BOS,153,180,140,155,333,295,0.861111,0.915033,0.885886,-1.542282,0.123005
27,Allen,MEM,189,168,143,134,357,277,0.797619,0.756614,0.775910,0.927416,0.353710
33,Almond,UTA,16,16,13,12,32,25,0.750000,0.812500,0.781250,-0.427618,0.668929


In [14]:
canvas = tp.Canvas(800, 300)
ax1 = canvas.axes(grid=(1, 2, 0), label="Histogram p-values")
hist_p = ax1.bars(np.histogram(data["pval"], bins=50, normed=True), color="steelblue")
hisp_p_density = ax1.plot([0, 1], [1, 1], color="red")
ax2 = canvas.axes(grid=(1, 2, 1), label="Histogram z-values")
hist_z = ax2.bars(np.histogram(data["zval"], bins=50, normed=True), color="orange")
x = np.linspace(-3, 3, 200)
hisp_z_density = ax2.plot(x, sp.stats.norm.pdf(x), color="red")

0.0 0.5 1.0 0.0 0.5 1.0 1.5 2.0 Histogram p-values <path d="M 457.2688557350981 247.88033199692481 L 458.68836501124184 247.68071905200827 L 460.10787428738558 247.46461408339192 L 461.52738356352933 247.23089143080742 L 462.94689283967313 246.97837134517059 L 464.36640211581687 246.70581948561852 L 465.78591139196067 246.41194662957702 L 467.20542066810441 246.09540861692602 L 468.62492994424815 245.75480654968618 L 470.04443922039195 245.38868726887566 L 471.46394849653569 244.99554413026516 L 472.88345777267944 244.57381810068421 L 474.30296704882323 244.12189919628253 L 475.72247632496709 243.63812828372008 L 477.14198560111078 243.12079926463346 L 478.56149487725452 242.56816166289144 L 479.98100415339832 241.97842363310485 L 481.40051342954206 241.34975540757728 L 482.82002270568591 240.68029319737639 L 484.23953198182966 239.96814356145418 L 485.6590412579734 239.21138825575318 L 487.0785505341172 238.40808957199562 L 488.49805981026094 237.55629617336837 L 489.91756908640468 236.65404943158507 L 491.33707836254848 235.69939026684003 L 492.75658763869222 234.69036648896565 L 494.17609691483597 233.62504063468157 L 495.59560619097977 232.50149829219109 L 497.01511546712351 231.31785690055202 L 498.43462474326725 230.07227500724747 L 499.85413401941099 228.762961963224 L 501.27364329555473 227.38818803038092 L 502.69315257169853 225.94629487210369 L 504.11266184784233 224.43570639297792 L 505.53217112398613 222.85493988931964 L 506.95168040012987 221.20261746765578 L 508.37118967627367 219.47747768382365 L 509.79069895241742 217.67838735096032 L 511.21020822856121 215.80435346037908 L 512.6297175047049 213.85453515520797 L 514.0492267808487 211.82825569275116 L 515.4687360569925 209.7250143278647 L 516.88824533313618 207.54449804626387 L 518.30775460927998 205.28659307364381 L 519.72726388542378 202.95139608384028 L 521.14677316156758 200.53922502703338 L 522.56628243771127 198.05062949723762 L 523.98579171385506 195.48640055707421 L 525.40530098999875 192.84757993711673 L 526.82481026614255 190.13546852698013 L 528.24431954228635 187.35163407580791 L 529.66382881843015 184.49791802093856 L 531.08333809457395 181.57644136531357 L 532.50284737071763 178.58960952664629 L 533.92235664686143 175.54011608451648 L 535.34186592300523 172.4309453553887 L 536.76137519914892 169.26537373007992 L 538.18088447529271 166.04696971340991 L 539.60039375143651 162.77959261164779 L 541.01990302758031 159.46738981989316 L 542.439412303724 156.11479266868011 L 543.85892157986768 152.72651079682561 L 545.27843085601148 149.30752502582246 L 546.69794013215528 145.8630787198548 L 548.11744940829908 142.39866762473093 L 549.53695868444288 138.92002818863043 L 550.95646796058656 135.43312437747849 L 552.37597723673036 131.94413300792237 L 553.79548651287405 128.45942763121184 L 555.21499578901785 124.98556101170564 L 556.63450506516165 121.52924625414521 L 558.05401434130545 118.09733664417715 L 559.47352361744925 114.69680427677538 L 560.89303289359293 111.33471755712317 L 562.31254216973673 108.01821766807646 L 563.73205144588042 104.75449410745374 L 565.15156072202421 101.55075940699659 L 566.57106999816801 98.414223152834438 L 567.99057927431181 95.352065434589832 L 569.41008855045561 92.371409856791203 L 570.82959782659941 89.479296251961074 L 572.2491071027431 86.682653239538126 L 573.66861637888678 83.988270778627026 L 575.08812565503058 81.402772865391128 L 576.50763493117438 78.932590527671522 L 577.92714420731818 76.583935270097484 L 579.34665348346186 74.362773122522526 L 580.76616275960566 72.274799443066314 L 582.18567203574946 70.32541462435708 L 583.60518131189326 68.519700847760177 L 585.02469058803706 66.862400025464723 L 586.44419986418075 65.357893064303269 L 587.86370914032454 64.01018057814008 L 589.28321841646823 62.822865167629431 L 590.70272769261203 61.799135377167126 L 592.12223696875583 60.941751429010665 L 593.54174624489951 60.253032823890024 L 594.96125552104331 59.734847886062695 L 596.38076479718711 59.388605318765

Используем метода Фишера:

In [9]:
T = -2 * np.sum(np.log(data["pval"]))
print 'p-value Fisher: {:.3e}'.format(1 - sp.stats.chi2.cdf(T, 2*len(data)))

p-value Fisher: 1.074e-04


Что будет, если мы применим поправку Бонферрони?

**Последовательная проверка**

In [10]:
from statsmodels.sandbox.stats.multicomp import multipletests

In [11]:
alpha = 0.05
data["reject_bc"] = 1*(data["pval"] < alpha / len(data))
print 'Number of rejections: {}'.format(data["reject_bc"].sum())

Number of rejections: 0


По сути, тест Бонферрони в данном сулчае эквивалентен одноименно методу при одновременной проверке, что и потдтверждается результатами.

In [12]:
is_reject, corrected_pvals, _, _ = multipletests(data["pval"], alpha=0.1, method='fdr_bh')

In [13]:
data["reject_fdr"] = 1*is_reject
data["pval_fdr"] = corrected_pvals
print 'Number of rejections: {}'.format(data["reject_fdr"].sum())

Number of rejections: 0


Несмотря на то, что тест асимптотически более мощный, ни одну индивидуальную гипотезу отвергнуть мы не смогли.